In [107]:
import os
import numpy as np
import pandas as pd
import string, re
import json

import nltk
from nltk.corpus import stopwords

from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize


In [2]:
os.chdir("..")
INTER_PATH = os.path.join("data", "interim")
with open(os.path.join(INTER_PATH, "pdf_files.json")) as json_file:
    data = json.load(json_file)
    
#data.keys()

dict_keys(['2019CVE 1713470_Chile', 'Decreto 51_Chile', 'Decreto 95_Chile', 'Decreto8_Chile', 'Decreto203_Chile', 'Ley 20412_Chile', 'LEY-SOBRE-CHILE', 'NormativaForestal_Chile', 'Programas de recuperacion_Chile', 'Resolution306_Chile', 'SUSTENTABILIDAD AGROAMBIENTAL_Chile', 'Constitution_ElSalvador', 'Decreto233_ElSalvador', 'Decreto272_Cafe', 'Decreto864_ApoyoCafe', 'educacion_ambiental_el_salvador', 'Ley de areas naturales protegidas_ElSalvador', 'Ley del medio ambiente_ElSalvador', 'Ley Especial Cafe_ElSalvador', 'Ley Forestal_ElSalvador', 'Ley Turismo_ElSalvador', 'LEY_GANADERO_ElSalvador', 'LeyAGROINDUSTRIA_ElSalvador', 'LeyElectricidad_ElSalvador', 'lncentivosSistemaAgroforestalCafe_Salvador', 'PREP_ElSalvador', 'Decreto_51_PINPEP_Guatemala', 'Decreto_109_Guatemala', 'Decreto2-2015_PROBOSQUE_Guatemala', 'Decreto101-96_Ley Forestal_Guatemal', 'gua60538', 'Resolucion_1.01-2007_Guatemala', 'Resolucion_1.2-98_Guatemala', 'Resolucion_1.30_Montos_Guatemala', 'Resolucion_2.43_Guatemala

In [3]:
spa_stopwords = set(stopwords.words('spanish'))

In [4]:
def most_freq_words(doc_list, stopwords_set, num_freq_words):
    filtered_words = list()
    for doc in doc_list:
        tokens = [word.lower() for word in word_tokenize(doc)]
        filtered_words.extend([word for word in tokens if word.isalpha() and word not in stopwords_set and len(word) > 1])
    
    return FreqDist(filtered_words).most_common(num_freq_words)

In [5]:
def docs_from_json(data):
    doc_list = list()
    for doc_json in data.values():
        doc_list.append(doc_json["Text"])
    
    return doc_list

In [6]:
all_docs = docs_from_json(data)

In [7]:
spa_stopwords.add('ley')
spa_stopwords.add('artículo')
freq_words_map = most_freq_words(all_docs, spa_stopwords, 50)

In [8]:
freq_words_map

[('forestal', 3966),
 ('manejo', 3124),
 ('nacional', 3028),
 ('ambiental', 2808),
 ('forestales', 2738),
 ('recursos', 2668),
 ('desarrollo', 2060),
 ('actividades', 1887),
 ('naturales', 1886),
 ('fauna', 1870),
 ('áreas', 1621),
 ('silvestre', 1594),
 ('caso', 1578),
 ('ser', 1524),
 ('ambiente', 1491),
 ('uso', 1434),
 ('plan', 1389),
 ('presente', 1370),
 ('gestión', 1361),
 ('conservación', 1303),
 ('ambientales', 1303),
 ('servicios', 1302),
 ('así', 1298),
 ('especies', 1294),
 ('deberá', 1269),
 ('reglamento', 1228),
 ('aprovechamiento', 1201),
 ('acuerdo', 1166),
 ('programa', 1165),
 ('cambio', 1145),
 ('ministerio', 1134),
 ('información', 1102),
 ('general', 1081),
 ('sistema', 1067),
 ('protección', 1048),
 ('siguientes', 995),
 ('dentro', 972),
 ('cada', 966),
 ('productos', 961),
 ('plazo', 954),
 ('medio', 948),
 ('según', 934),
 ('decreto', 926),
 ('apoyo', 923),
 ('acciones', 917),
 ('regional', 916),
 ('autoridad', 908),
 ('planes', 902),
 ('evaluación', 883),
 ('se

In [23]:
sents = [sent.lower() for sent in sent_tokenize(sample_txt)]
filtered_sents = [" ".join([word for word in sent.split() if word.isalpha() and word not in spa_stopwords]) for sent in sents]

NameError: name 'sample_txt' is not defined

In [69]:
FreqDist(filtered_sents)

FreqDist({'': 45, 'modificado letra': 17, 'reemplazado': 16, 'ministerio publicado': 15, 'promulgado publicado': 10, 'modificado': 10, 'sustituido letra': 10, 'reemplazado letra': 7, 'agregado letra': 5, 'agregado': 5, ...})

### 1. Data Cleaning

- 'CreditoGanadero_Mexico': not readable

In [129]:
stopwords_spanish = set(stopwords.words('spanish')).union(set(['ley', 'artículo']))

In [108]:
puntuation = [p for p in set(string.punctuation) if p not in (".")]

In [130]:
def clean_stopwords(df):
    df = df.lower().replace("\n", " ").replace("\t", " ").strip(" ")
    df = "".join( c for c in df.split(" ") if c not in puntuation)
    df = " ".join([w for w in df.split() if w not in stopwords_spanish])
    #df = " ".join([w for w in df.split() if w not in additional_stopwords])
    return df

In [90]:
# From Json to DF
clean_docs = pd.DataFrame([])
for doc_name, doc_json in data.items():
    doc = pd.json_normalize(doc_json)
    doc.insert(0, "Document", doc_name) 
    #print(doc)
    clean_docs = clean_docs.append(doc, ignore_index = True)


In [206]:
# Set label
clean_docs.insert(11,'Label','Relevant')

### Lexical Analysis

In [123]:
corpus = clean_docs.Text

In [126]:
all_words = [w.split() for w in corpus]

all_flat_words = [ewords for words in all_words for ewords in words]

In [133]:
#removing all the stop words from the corpus
all_flat_words_ns = [w for w in all_flat_words if w not in stopwords_spanish]

#removing all duplicates
set_nf = set(all_flat_words_ns)

In [134]:
print("Number of unique vocabulary words in the text_description column of the dataframe: %d"%len(set_nf))

Number of unique vocabulary words in the text_description column of the dataframe: 79262


### Data Pre-Processing

The following steps are performed:
   - Converting all of the data into lower case.
   - FInd the root of the words to further reduce the feature size

to do: When the document have 'Keywords', the keywords are added to the description.


In [142]:
from nltk import word_tokenize
from nltk.stem import SnowballStemmer

In [192]:
porter = SnowballStemmer('spanish')
#porter=nltk.PorterStemmer()

for each_row in clean_docs.itertuples():
    #m1=map(lambda x: x , (str(each_row[4])+' '+str(each_row[10])).lower().split())
    m1 = map(lambda x: x , (each_row[11]).lower().split())
    #print(each_row[11])
    #Using Porter Stemmer in NLTK, find root
    m2 = map(lambda x: porter.stem(x), m1)
    #pre-processed string is stored in new column
    clean_docs.loc[each_row[0],'Desc'] = ' '.join(m2)

In [196]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [197]:
# New corpus
corpus = clean_docs['Desc']
#Initializing TFIDF vectorizer to conver the raw corpus to a matrix of TFIDF features 
# and also enabling the removal of stopwords
vectorizer = TfidfVectorizer(stop_words=stopwords_spanish)

In [198]:
#Creating TFIDF features sparse matrix by fitting it on the specified corpus 
tfidf_matrix = vectorizer.fit_transform(corpus).todense()

#Grabbing the name of the features.
tfidf_names = vectorizer.get_feature_names()

In [202]:
print("Number of TF-IDF Features: ", tfidf_matrix.shape[1])

# There are  34,617 columns that will be used for training the classifier
# These are much smaller than the total number of unique vocabulary words
# (79,262) that previously calculated

Number of TFIDF Features:  34617


In [203]:
training_time_container={'b_naive_bayes':0,'mn_naive_bayes':0,'random_forest':0,'linear_svm':0}
prediction_time_container={'b_naive_bayes':0,'mn_naive_bayes':0,'random_forest':0,'linear_svm':0}

accuracy_container={'b_naive_bayes':0,'mn_naive_bayes':0,'random_forest':0,'linear_svm':0}

### Classifiers

First, split our existing dataset into training and test data.


In [237]:
from time import time

import sklearn.metrics
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import classification_report


In [212]:
# Split dataset into training and test data (70-30 ratio)

#considering the TFIDF features as the input
variables = tfidf_matrix
#labels for the classifier
labels = clean_docs.Label
#splitting the data
var_train, var_test, labels_train, labels_test = train_test_split(variables, labels, test_size=.3)

In [213]:
#analyzing the shape of the training and test data-set:
print('Shape of Training Data: '+str(variables_train.shape))
print('Shape of Test Data: '+str(variables_test.shape))

Shape of Training Data: (44, 34617)
Shape of Test Data: (20, 34617)


#### Naive Bayes

Naive Bayes is one of the most widely used classification algorithm in text mining applications. The main assumption is that all the features are independent of each other. The condition of independence may not be valid in many circumstances but as a base line model, its a good starting point.

Naive Bayes uses the probabilities of each attribute belonging to each class to make a prediction. There are two forms of Naive Bayes:

   1. Bernoulli, designed for boolean/binary features i.e. just considers the presence or absense of a feature.
   2. Multinomial, which also considers the occurrence counts of the feature.

We will apply both and then will assess their respective accuracy scores.

In [214]:
#Define classifier
bnb_classifier = BernoulliNB()

t0 = time()
bnb_classifier = bnb_classifier.fit(var_train, labels_train)
#Training the classifier on the training data
training_time_container['b_naive_bayes'] = time()-t0


In [215]:
#Predictions
bnb_predictions = bnb_classifier.predict(var_test)
prediction_time_container['b_naive_bayes']=time()-t0

In [227]:

#Metrics
nb_ascore = sklearn.metrics.accuracy_score(labels_test, bnb_predictions)
accuracy_container['b_naive_bayes'] = nb_ascore
con_mat = pd.DataFrame(sklearn.metrics.confusion_matrix(labels_test, bnb_predictions))

print("Bernoulli Naive Bayes Accuracy Score: %f"%accuracy_container['b_naive_bayes'])
print("Training Time: %f"%training_time_container['b_naive_bayes'])
print("Prediction Time: %f"%prediction_time_container['b_naive_bayes'])
print("Confusion Matrix of Bernoulli Naive Bayes Classifier output: ")
con_mat.div(con_mat.sum(0), axis=1).round(2)*100

Bernoulli Naive Bayes Accuracy Score: 1.000000
Training Time: 0.092194
Prediction Time: 185.372773
Confusion Matrix of Bernoulli Naive Bayes Classifier output: 


,0
0,100.0


In [229]:
print("Classification Metrics: ")
print(sklearn.metrics.classification_report(labels_test,bnb_predictions))

Classification Metrics: 
              precision    recall  f1-score   support

    Relevant       1.00      1.00      1.00        20

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



#### Multinomial Naive Bayes

Bernoulli Naive Bayes only considers whether a feature is present or not. However, if we also take into account the occurrence weight or count of the feature as well (in our case, the TF-IDF weight of each feature), we can hypothesize that the performance of such classifier will be equally good, if not better. That is assumption for the Multi-nomial Naive Bayes.

In [230]:
mn_bayes = MultinomialNB()
t0 = time()
mn_bayes_fit = mn_bayes.fit(var_train,labels_train)
training_time_container['mn_naive_bayes'] = time()-t0
t0 = time()

prediction_mn = mn_bayes_fit.predict(variables_test)
prediction_time_container['mn_naive_bayes'] = time()-t0
mn_ascore = sklearn.metrics.accuracy_score(labels_test, prediction_mn) 
accuracy_container['mn_naive_bayes'] = mn_ascore

In [231]:
print("Accuracy Score of Multi-Nomial Naive Bayes: %f" %(mn_ascore))
print("Training Time: %fs"%training_time_container['mn_naive_bayes'])
print("Prediction Time: %fs"%prediction_time_container['mn_naive_bayes'])

Accuracy Score of Multi-Nomial Naive Bayes: 1.000000
Training Time: 0.027730s
Prediction Time: 0.003846s


### Random Forest Classifier

In Random Forest, a subset of the training data is fit on a number of decision trees. Random Forests have the characteristic to minimize variance if its there in the data-set. 

In [232]:
rf_classifier = RandomForestClassifier(n_estimators=50)
t0 = time()
rf_classifier = rf_classifier.fit(var_train, labels_train)

training_time_container['random_forest'] = time()-t0
print("Training Time: %fs"%training_time_container['random_forest'])

t0 = time()
rf_predictions = rf_classifier.predict(var_test)
prediction_time_container['random_forest'] = time()-t0
print("Prediction Time: %fs"%prediction_time_container['random_forest'])

accuracy_container['random_forest'] = sklearn.metrics.accuracy_score(labels_test, rf_predictions)
print ("Accuracy Score of Random Forests Classifier: ")
print(accuracy_container['random_forest'])
print(sklearn.metrics.confusion_matrix(labels_test, rf_predictions))

Training Time: 0.306189s
Prediction Time: 0.025298s
Accuracy Score of Random Forests Classifier: 
1.0
[[20]]


In [239]:
classifier = RandomForestClassifier()

# Hyperparameters
hyper_param_grid = {'n_estimators': [10, 50, 100], 
                     'max_depth': [5,10,20,50,100], 
                     'max_features': ['sqrt','log2'],
                     'min_samples_split': [2,5,10,20,50]}

# Gid search
grid_search = GridSearchCV(classifier, 
                           hyper_param_grid, 
                           scoring = 'accuracy',
                           cv = 5, 
                           n_jobs = -1,
                           verbose = 3)
grid_search.fit(var_train, labels_train)

# Best params
grid_search.best_params_

# Best score
print('Valor de la mejor métrica para el modelo de Random Forest:', grid_search.best_score_ * 100)

Fitting 5 folds for each of 150 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   21.6s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   43.3s
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:  1.2min


Valor de la mejor métrica para el modelo de Random Forest: 100.0


[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:  1.8min finished
